In [ ]:
import os
from PIL import Image
import numpy as np
import os
import cv2
from matplotlib import pyplot as plt
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.applications.vgg16 import decode_predictions
from tensorflow.python.keras import layers
from tensorflow.python.keras import models
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.backend import get_session



class DeepModel:
    
    def __init__(self):
        np.random.seed(42)
        tf.set_random_seed(42)
        self.sess = tf.Session()
        set_session(self.sess)
        cnnModel = models.Sequential()

        cnnModel.add(layers.Conv2D(10, (3,3), activation="relu", 
                                   input_shape=(120,120,3)))

        cnnModel.add(layers.MaxPooling2D((2,2)))

        cnnModel.add(layers.Conv2D(64, (3,3), activation="relu"))

        cnnModel.add(layers.MaxPooling2D((2,2)))

        cnnModel.add(layers.Conv2D(64, (3,3), activation="relu"))

        cnnModel.add(layers.Flatten())

        cnnModel.add(layers.Dense(64, activation="relu"))

        cnnModel.add(layers.Dense(32, activation="relu"))

        cnnModel.add(layers.Dense(3, activation="softmax"))

        cnnModel.compile(optimizer="adam", 
                        loss="categorical_crossentropy",
                        metrics=["accuracy"])
        
        cnnModel.load_weights('./my_checkpoint')
        self.model=cnnModel
        self.graph = tf.get_default_graph()

    def predict(self,data_file):
        with self.graph.as_default():
            session = get_session()
            init = tf.global_variables_initializer()
            session.run(init)
            X_i = Image.open(os.path.join(data_file))
            X_i = np.array(X_i.resize((120,120))) / 255.0
            X_i=X_i.reshape([-1,120,120,3])
            return (self.model.predict_classes(X_i))

    
from flask import Flask, render_template, request
from werkzeug import secure_filename
app = Flask(__name__)

@app.route('/uploader', methods = ['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        f = request.files['file']
        f.save(secure_filename("qry.png"))
        file_name='qry.png'
        c=model.predict(file_name)
        if c[0]==0:
            return 'melon'
        else:
            return 'lemon'
    
    
@app.route('/test', methods = ['GET', 'POST'])
def test():
    if request.method == 'GET':
        return 'Hello World'
    
if __name__ == '__main__':
    model=DeepModel()
    app.run(host='0.0.0.0',debug = False,threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Jan/2020 15:25:47] "POST /uploader HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2020 15:25:54] "POST /uploader HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2020 15:26:04] "POST /uploader HTTP/1.1" 200 -
